In [1]:
import sys, os
sys.path.insert(0, "/home/storage/hans/jax_reco_new")
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import jax.numpy as jnp
from jax.scipy import optimize
import jax
jax.config.update("jax_enable_x64", True)
import optimistix as optx
import quadax

from jax.scipy.special import gammaincc, erf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lib.network import get_network_eval_fn
from lib.trafos import transform_network_outputs, transform_network_inputs
from lib.cgamma import c_multi_gamma_prob, c_multi_gamma_sf, c_multi_gamma_sf_approx
from lib.cgamma_biweight_v3 import c_multi_gamma_biweight_mpe_logprob
from lib.plotting import adjust_plot_1d

from dom_track_eval import get_eval_network_doms_and_track

In [2]:
eval_network = get_network_eval_fn(bpath='/home/storage/hans/jax_reco_new/data/network')

In [3]:
dist = 2
z = -210
rho = 0.0
zenith = np.pi/2
azimuth = 0.0

x = jnp.array([dist, rho, z, zenith, azimuth])
x_prime = transform_network_inputs(x)
y = eval_network(x_prime)
logits, gamma_as, gamma_bs = transform_network_outputs(y)
mix_probs = jax.nn.softmax(logits)

g_a = gamma_as
g_b = gamma_bs
sigma = 3

print(gamma_as, gamma_bs)

[1.00045562 1.2639036  1.00001584] [0.04895381 1.50627305 0.00372613]


In [4]:
sigma = 3.0
c_multi_gamma_biweight_mpe_logprob(0.0, mix_probs, g_a, g_b, 10.0, sigma)

Array(-4.24553381, dtype=float64)

In [5]:
from lib.cgamma_biweight_v2 import c_multi_gamma_biweight_prob, c_multi_gamma_biweight_cdf

def c_multi_gamma_biweight_mpe_logprob_orig(x, mix_probs, a, b, n_photons, sigma=3.0):
    # share the two evaluations of the upper incomplete gamma function.
    # across all subsequent function calls to avoid
    # costly re-evaluations.
    prob = c_multi_gamma_biweight_prob(x, mix_probs, a, b, sigma)
    cdf = c_multi_gamma_biweight_cdf(x, mix_probs, a, b, sigma)
    return jnp.log(n_photons) + jnp.log(prob) + (n_photons-1.0) * jnp.log(1.0-cdf)

In [6]:
c_multi_gamma_biweight_mpe_logprob_orig(0.0, mix_probs, g_a, g_b, 10.0, sigma)

Array(-4.24553381, dtype=float64)

In [7]:
f = jax.grad(c_multi_gamma_biweight_mpe_logprob, argnums=(2, 3))

In [8]:
f(0.0, mix_probs, g_a, g_b, 10.0, sigma)

(Array([0.15164324, 0.78358592, 0.00715522], dtype=float64),
 Array([-1.20103767, -0.63101636, -0.39052533], dtype=float64))